In [1]:
import torch
from pathlib import Path
from tqdm import tqdm
from PIL import Image
from transformers import BlipForQuestionAnswering, BlipProcessor
from torch.utils.data import DataLoader, Dataset
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

c:\Users\CSE IIT BHILAI\Downloads\dlcv\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAX_LENGTH=20
BATCH_SIZE=8
NUM_EPOCHS=20
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
BASE_DIR=Path(r'c:\Users\CSE IIT BHILAI\Downloads\dlcv')
%cd {BASE_DIR}

cuda
c:\Users\CSE IIT BHILAI\Downloads\dlcv


c:\Users\CSE IIT BHILAI\Downloads\dlcv\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
torch.cuda.is_available()

True

In [4]:
model_name = "Salesforce/blip-vqa-base"
model = BlipForQuestionAnswering.from_pretrained(model_name)
processor = BlipProcessor.from_pretrained(model_name)

In [5]:

import json

# Function to extract text from image
def extract_text(image_path):
    doc = DocumentFile.from_images(image_path)
    predictor = ocr_predictor(pretrained=True)
    result = predictor(doc)
    json_export = result.export()
    fields_to_remove = ['objectness_score','crop_orientation','confidence','geometry', 'page_idx', 'dimensions', 'orientation', 'language', 'artefacts']
    remove_fields(json_export, fields_to_remove)
    for page in json_export['pages']:
        for block in page['blocks']:
            if 'geometry' in block:
                del block['geometry']
            for line in block.get('lines', []):
                if 'geometry' in line:
                    del line['geometry']
    
    modified_json = json.dumps(json_export, separators=(',', ':'))
    final_str = extract_values_to_string(modified_json)
    return final_str


def remove_fields(obj, fields):
    if isinstance(obj, list):
        for item in obj:
            remove_fields(item, fields)
    elif isinstance(obj, dict):
        for key in list(obj.keys()):
            if key in fields:
                del obj[key]
            else:
                remove_fields(obj[key], fields)
    
def remove_geometry(data):
    if isinstance(data, list):
        for item in data:
            remove_geometry(item)
    elif isinstance(data, dict):
        if 'geometry' in data:
            del data['geometry']
        for key, value in data.items():
            remove_geometry(value)

def extract_values_to_string(json_string):
    # Recursive function to extract 'value' fields
    data = json.loads(json_string)
    def extract_values(block):
        values = []
        if isinstance(block, dict):
            for key, value in block.items():
                if key == "value":
                    values.append(value)
                else:
                    values.extend(extract_values(value))
        elif isinstance(block, list):
            for item in block:
                values.extend(extract_values(item))
        return values
    
    # Extract values and combine them into a single string
    values = extract_values(data)
    return ' '.join(values)

In [6]:
w = extract_text(r'C:\Users\CSE IIT BHILAI\Downloads\dlcv\images\test\21IYQostduL.jpg')
print(w)

c:\Users\CSE IIT BHILAI\Downloads\dlcv\.venv\Lib\site-packages\doctr\models\utils\pytorch.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(archiv

USB Rechargeable ult-in 400mah Kechargeable -Battery a 60 days One-time Full-chargod - Under the condons: ghts up < times/day,


In [7]:
from torch.utils.data import Dataset
from PIL import Image
class MultiModalVQADataset(Dataset):
    def __init__(self, dataframe, processor, max_length=MAX_LENGTH):
        self.dataframe = dataframe
        self.processor = processor
        # self.ocr_reader = ocr_reader
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        question = row["question"]
        answer = row["answer"]

        # Extract text from the image
        ocr_text = extract_text(row["image_path"])

        # Prepare inputs for the vision model
        inputs = self.processor(
            text=question,
            images=image,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        inputs["labels"] = self.processor.tokenizer(
            answer,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ).input_ids.squeeze(0)

        for key in inputs:
            inputs[key]=inputs[key].squeeze(0)
        inputs["ocr_text"] = ocr_text
        

        return inputs

In [8]:
import pandas as pd

df = pd.read_csv(r"C:\Users\CSE IIT BHILAI\Downloads\dlcv\data\train_vqa.csv")
train_dataset = MultiModalVQADataset(df.iloc[:1000], processor)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()


In [11]:
import os

ckpt_dir = "./checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)

In [12]:
import torch
from torch import nn
from transformers import BlipForConditionalGeneration,BlipForQuestionAnswering, AutoTokenizer, AutoModel


class MultiModalVQAModel(nn.Module):
    def __init__(self, vision_model_name, text_model_name, hidden_size=768, max_length=20):
        super(MultiModalVQAModel, self).__init__()

        # Vision model (BLIP)
        self.vision_model = BlipForConditionalGeneration.from_pretrained(vision_model_name)

        # Language model for OCR-extracted text
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(text_model_name)
        # vocab_size = self.vision_model.text_decoder.config.vocab_size
        # Fusion Layer
        self.fusion_layer = nn.Linear(hidden_size * 2, hidden_size)
        self.max_length = max_length
        # Output Layer

    def forward(self, pixel_values, input_ids, attention_mask, ocr_text, labels=None):
        # Vision model outputs
        # pixel_values = pixel_values.squeeze(1)
        vision_outputs = self.vision_model.vision_model(
            pixel_values=pixel_values
        ).last_hidden_state
        # vision_features = vision_outputs.last_hidden_state.mean(dim=1)
        # Text model outputs (OCR text)
        ocr_inputs = self.tokenizer(
            ocr_text, return_tensors="pt", padding=True, truncation=True
        ).to(device)
        ocr_outputs = self.text_model(**ocr_inputs).last_hidden_state.mean(
            dim=1
        ).to(device)  # Mean-pooling for text features

        # Fuse vision and OCR features
        combined_features = torch.cat(
            [vision_outputs.mean(dim=1), ocr_outputs], dim=1
        )  # Mean-pooling image features
        fused_features = self.fusion_layer(combined_features)
        decoder_inputs = {
            "pixel_values": pixel_values,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "encoder_outputs": (fused_features.unsqueeze(1),),  # Treat fused features as encoder output
            "labels": labels,
        }
        outputs = self.vision_model(**decoder_inputs)
        # Predict answer
        # out = self.lin(fused_features)
        # logits = self.classifier(out)
        return outputs

In [13]:
vision_model_name = "Salesforce/blip-vqa-base"
text_model_name = "bert-base-uncased"
model = MultiModalVQAModel(vision_model_name, text_model_name).to(device)

In [14]:
num_epochs = NUM_EPOCHS
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader):
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        ocr_text = batch["ocr_text"]  # OCR text is a list of strings

        # Forward pass
        logits = model(
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            input_ids=input_ids,
            ocr_text=ocr_text,
        )

        # Compute loss
        loss = criterion(logits.to(torch.float), labels.to(float))
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {avg_loss}")
    checkpoint_path = os.path.join(ckpt_dir, f"epoch_{epoch+1}.pth")
    torch.save(
        {
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": avg_loss,
        },
        checkpoint_path,
    )
    print(f"Checkpoint saved at {checkpoint_path}")
    print()

  0%|          | 0/125 [00:00<?, ?it/s]c:\Users\CSE IIT BHILAI\Downloads\dlcv\.venv\Lib\site-packages\doctr\models\utils\pytorch.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fe

Epoch 1/20, Loss: 78101.85238684868
Checkpoint saved at ./checkpoints\epoch_1.pth



100%|██████████| 125/125 [34:33<00:00, 16.59s/it]


Epoch 2/20, Loss: 78105.21723820567
Checkpoint saved at ./checkpoints\epoch_2.pth



100%|██████████| 125/125 [33:54<00:00, 16.28s/it]


...

Epoch 18/20, Loss: 78105.39428282618
Checkpoint saved at ./checkpoints\epoch_18.pth



100%|██████████| 125/125 [33:58<00:00, 16.31s/it]


Epoch 19/20, Loss: 78094.86474051165
Checkpoint saved at ./checkpoints\epoch_19.pth



100%|██████████| 125/125 [33:59<00:00, 16.31s/it]


Epoch 20/20, Loss: 78091.38830610848
Checkpoint saved at ./checkpoints\epoch_20.pth



In [20]:
save_path = f"{BASE_DIR}\\models\\custom_blip_model"
# model.save_pretrained(save_path)
# processor.save_pretrained(save_path)

In [22]:
torch.save(model.state_dict(), f"{save_path}\\model.pth")

In [44]:
class MultiModalTestVQADataset(Dataset):
    def __init__(self, dataframe, processor, max_length=MAX_LENGTH):
        self.dataframe = dataframe
        self.processor = processor
        # self.ocr_reader = ocr_reader
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        question = row["question"]
        answer = row["answer"]

        # Extract text from the image
        ocr_text = extract_text(row["image_path"])

        # Prepare inputs for the vision model
        inputs = self.processor(
            text=question,
            images=image,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        inputs = {key: value.squeeze(0) if isinstance(value, torch.Tensor) else value for key, value in inputs.items()}

        
        for key in inputs:
            inputs[key] = inputs[key].squeeze(0)
            
        inputs["ocr_text"] = ocr_text
        return {
            "pixel_values": inputs["pixel_values"],
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "ocr_text" : inputs["ocr_text"],
            "question": question,
            "answer": answer,  # For evaluation purposes
        }

In [45]:
test_df = pd.read_csv(f"{BASE_DIR}/data/test_vqa.csv")

In [46]:
test_dataset = MultiModalTestVQADataset(test_df, processor)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [47]:
# model = BlipForQuestionAnswering.from_pretrained(save_path)
# processor = BlipProcessor.from_pretrained(save_path)

In [48]:
from evaluate import load

metric = load("sacrebleu")

In [ ]:
model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in test_dataloader:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        ocr_text = batch["ocr_text"]

        # Generate predictions
        logits = model(
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            input_ids=input_ids,
            ocr_text=ocr_text,
        )
        predicted_ids = torch.argmax(logits, dim=1)

        # Decode predictions
        predicted_answers = processor.tokenizer.batch_decode(
            predicted_ids, skip_special_tokens=True
        )
        ground_truth_answer = batch["answer"]

        predictions.append(predicted_answers)
        references.append(ground_truth_answer)

        metric.add_batch(predictions=predicted_answers, references=ground_truth_answer)

results = metric.compute()
print(f"BLEU Score: {results['score']}")

c:\Users\CSE IIT BHILAI\Downloads\dlcv\.venv\Lib\site-packages\doctr\models\utils\pytorch.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(archiv

In [ ]:
import numpy as np
from itertools import chain

In [ ]:
results_df = pd.DataFrame()
results_df["labels"] = list(chain.from_iterable(references))
results_df["predictions"] = list(chain.from_iterable(predictions))
results_df.head()

In [ ]:
results_df.to_csv(f"{BASE_DIR}/results/custom_blip.csv", index=False)

In [ ]:
import torch
from torch import nn
from transformers import BlipForConditionalGeneration, AutoTokenizer, AutoModel

class MultiModalVQAModelWithDecoder(nn.Module):
    def _init_(self, vision_model_name, text_model_name, hidden_size=768, max_length=128):
        """
        A multimodal VQA model combining image and OCR text features,
        generating full sentences as answers using a BLIP decoder.

        Args:
            vision_model_name (str): Pretrained vision model (e.g., BLIP).
            text_model_name (str): Pretrained text model (e.g., BERT or RoBERTa).
            hidden_size (int): Size of the hidden layer for fusion.
            max_length (int): Maximum length of generated answers.
        """
        super(MultiModalVQAModelWithDecoder, self)._init_()

        # Vision model (BLIP for image encoding)
        self.vision_model = BlipForConditionalGeneration.from_pretrained(vision_model_name)

        # Text model for OCR-extracted text (e.g., BERT or RoBERTa)
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(text_model_name)

        # Fusion Layer: Combines vision and text features
        self.fusion_layer = nn.Linear(hidden_size * 2, hidden_size)

        # Max length for decoder-generated answers
        self.max_length = max_length

    def forward(self, pixel_values, input_ids, attention_mask, ocr_text, labels=None):
        """
        Forward pass of the model.

        Args:
            pixel_values (torch.Tensor): Preprocessed image tensor.
            input_ids (torch.Tensor): Tokenized question input IDs.
            attention_mask (torch.Tensor): Attention mask for the question.
            ocr_text (list of str): OCR-extracted text from images.
            labels (torch.Tensor, optional): Tokenized ground-truth answer IDs for training.

        Returns:
            torch.Tensor: Loss (if labels are provided) or logits (during inference).
        """
        # Vision features from BLIP
        vision_outputs = self.vision_model.vision_model(pixel_values=pixel_values)
        vision_features = vision_outputs.last_hidden_state.mean(dim=1)  # Mean pooling

        # Text features from OCR-extracted text
        ocr_inputs = self.tokenizer(
            ocr_text, padding=True, truncation=True, return_tensors="pt"
        )
        ocr_outputs = self.text_model(**ocr_inputs.to(vision_features.device))
        ocr_features = ocr_outputs.last_hidden_state.mean(dim=1)  # Mean pooling

        # Fusion of vision and OCR features
        combined_features = torch.cat([vision_features, ocr_features], dim=1)
        fused_features = self.fusion_layer(combined_features)

        # Pass fused features to BLIP's decoder
        decoder_inputs = {
            "pixel_values": pixel_values,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "encoder_outputs": (fused_features.unsqueeze(1),),  # Treat fused features as encoder output
            "labels": labels,
        }
        outputs = self.vision_model(**decoder_inputs)

        return outputs